In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, BatchNormalization, Add, Activation, AveragePooling2D, Dropout, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import categorical_crossentropy

In [ ]:
def load_dataset(dataset):
  if dataset == 'mnist':
    return tf.keras.datasets.mnist.load_data()
  elif dataset == 'fashion_mnist':
    return tf.keras.datasets.fashion_mnist.load_data()
  elif dataset == 'cifar-10':
    return tf.keras.datasets.cifar10.load_data()
  else:
    return -1

In [ ]:
def preprocess_dataset(x_train, y_train, x_test, y_test):
  # expand the dimension (add the channel axis to MNIST and Fashion_MNIST)
  if len(x_train.shape) == 3 :
    x_train = tf.expand_dims(x_train, -1)
    x_test = tf.expand_dims(x_test, -1)

  # resize the image
  x_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((64,64))) for im in x_train])
  x_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((64,64))) for im in x_test])

  # Convert the labels to their one-hot representation
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  # Normalise the dataset by mean subtraction
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train_mean = np.mean(x_train, axis=0)
  x_train -= x_train_mean
  x_test -= x_train_mean

  return x_train, y_train, x_test, y_test

In [ ]:
def visualize(pred_labels, test_labels, test_images, dataset_name):
  # list to store the index of the success cases and fail cases
  success_cases = []
  fail_cases = []
  for i in range(len(test_labels)):
    if test_labels[i].argmax() == pred_labels[i].argmax() and len(success_cases)<5:
      success_cases.append(i)
    elif test_labels[i].argmax() != pred_labels[i].argmax() and len(fail_cases)<5:
      fail_cases.append(i)
    if len(success_cases) == 4 and len(fail_cases) == 4:
      break
  
  # cmap = gray from MNIST and fashion_mnist
  if len(test_images[0].shape) == 2:
    cmap = 'gray'
  else:
    cmap = None
  
  # plotting the success cases
  fig, ax = plt.subplots(nrows=2, ncols=2)
  for row in ax:
    for col in row:
        col.imshow(test_images[success_cases.pop(0)], interpolation='nearest', cmap=cmap)
  fig.suptitle('Success Cases of resnet34 on ' + dataset_name)

  # plotting the fail cases
  plt.show()
  fig, ax = plt.subplots(nrows=2, ncols=2)
  for row in ax:
    for col in row:
        col.imshow(test_images[fail_cases.pop(0)], interpolation='nearest', cmap=cmap)
  fig.suptitle('Fail Cases of resnet34 on ' + dataset_name)
  plt.show()

In [ ]:
def identity_block(x, f, dropout_rate):
  x_skip = x
  x = Conv2D(filters=f,kernel_size=(3,3),strides=(1, 1),padding="same", kernel_initializer=tf.initializers.GlorotUniform(),kernel_regularizer=l2(1e-4))(x)
  x = Dropout(dropout_rate)(x)
  x = BatchNormalization(axis=3)(x)
  # x = Activation('relu')(x)
  x = LeakyReLU()(x)
  x = Conv2D(filters=f,kernel_size=(3,3),strides=(1, 1),padding="same", kernel_initializer=tf.initializers.GlorotUniform(),kernel_regularizer=l2(1e-4))(x)
  x = Dropout(dropout_rate)(x)
  x = BatchNormalization(axis=3)(x)
  x = Add()([x, x_skip])
  # x = Activation('relu')(x)
  x = LeakyReLU()(x)
  return x

def conv_block(x, f, dropout_rate):
  x_skip = x
  x = Conv2D(filters=f,kernel_size=(3,3),strides=(2, 2),padding="same", kernel_initializer=tf.initializers.GlorotUniform(),kernel_regularizer=l2(1e-4))(x)
  x = Dropout(dropout_rate)(x)
  x = BatchNormalization(axis=3)(x)
  # x = Activation('relu')(x)
  x = LeakyReLU()(x)
  x = Conv2D(filters=f,kernel_size=(3,3),strides=(1, 1),padding="same", kernel_initializer=tf.initializers.GlorotUniform(),kernel_regularizer=l2(1e-4))(x)
  x = Dropout(dropout_rate)(x)
  x = BatchNormalization(axis=3)(x)
  x_skip = Conv2D(filters=f,kernel_size=(1,1),strides=(2, 2),padding="same", kernel_initializer=tf.initializers.GlorotUniform(),kernel_regularizer=l2(1e-4))(x_skip)
  x = Dropout(dropout_rate)(x)
  x_skip = BatchNormalization(axis=3)(x_skip)
  x = Add()([x, x_skip])
  # x = Activation('relu')(x)
  x = LeakyReLU()(x)
  return x

def model_resnet34(image_shape, num_category):
  filters = [64, 128, 256, 512]
  blocks = [3, 4, 6, 3]
  dropout_rate = 0.2
  inputs = Input(shape=image_shape)
  x = Conv2D(filters=64,kernel_size=(7,7),strides=(2, 2),padding="same",kernel_initializer=tf.initializers.GlorotUniform(),kernel_regularizer=l2(1e-4))(inputs)
  x = Dropout(dropout_rate)(x)
  x = BatchNormalization(axis=3)(x)
  # x = Activation('relu')(x)
  x = LeakyReLU()(x)
  x = MaxPooling2D((3, 3), strides=(2, 2), padding="same")(x)

  first_block = True
  for b in blocks:
    f = filters.pop(0)
    x = identity_block(x, f, dropout_rate) if first_block == True else conv_block(x, f, dropout_rate)
    first_block = False
    for _ in range(b-1):
      x = identity_block(x, f, dropout_rate)

  x = AveragePooling2D(pool_size=(2,2))(x)
  x= Flatten()(x)
  outputs = Dense(units=num_category, activation='softmax', kernel_initializer=tf.initializers.GlorotUniform())(x)
  model = Model(inputs=inputs, outputs=outputs)
  
  return model

In [ ]:
dataset_name = 'mnist'
(train_img, train_lbl), (test_img, test_lbl) = load_dataset(dataset_name)

In [ ]:
train_images, train_labels, test_images, test_labels = preprocess_dataset(train_img, train_lbl, test_img, test_lbl)

In [ ]:
augment_data = True
batch_size = 128
validation_percent = 0.1
epochs = 100
if augment_data:
  datagen = ImageDataGenerator(
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      rotation_range=0,
      fill_mode='nearest',
      validation_split=validation_percent
  )
  train_iterator = datagen.flow(train_images, train_labels, batch_size=batch_size, subset='training')
  validation_iterator = datagen.flow(train_images, train_labels, batch_size=batch_size, subset='validation')

In [ ]:
resnet34 = model_resnet34(image_shape = train_images[0].shape, num_category = len(train_labels[0]))
resnet34.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])
resnet34.summary()

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=3, min_lr=0.5e-6)
early_stop = EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True)
if augment_data:
  history = resnet34.fit(train_iterator, epochs=epochs, steps_per_epoch=len(train_iterator), 
                                validation_data=validation_iterator, validation_steps=len(validation_iterator), 
                                callbacks=[reduce_lr, early_stop])
else:
  history = resnet34.fit(x=train_images, y=train_labels, epochs=epochs, batch_size=batch_size, 
                      validation_split=validation_percent,
                      callbacks=[reduce_lr, early_stop])

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy of resnet34 on ' + dataset_name)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss of resnet34 on ' + dataset_name)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
eval = resnet34.evaluate(test_images, test_labels, batch_size=batch_size)

In [ ]:
predictions = resnet34.predict(test_images, batch_size=batch_size)

In [ ]:
visualize(predictions, test_labels, test_img, dataset_name)